In [1]:
import dill
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('upload_alpha.py'))))

import json

import gemini

import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

In [2]:
target_weight = {'USDCUSDT': 0.1,
                 'BCHUSDT' : 0.1,
                 'BTCUSDT' : 0.1,
                 'DOGEUSDT': 0.1,
                 'ETHUSDT' : 0.1,
                 'LTCUSDT' : 0.1,
                 'CRVUSDT' : 0.1,
                 'FTMUSDT' : 0.1,
                 'LINKUSDT': 0.1,
                 'SOLUSDT' : 0.1}

In [3]:
curr_ammount = {'USDCUSDT': 10,
                'BCHUSDT' : 0,
                'BTCUSDT' : 0,
                'DOGEUSDT': 0,
                'ETHUSDT' : 0,
                'LTCUSDT' : 0,
                'CRVUSDT' : 0,
                'FTMUSDT' : 0,
                'LINKUSDT': 0,
                'SOLUSDT' : 0}

In [4]:
with open('curr_ammount.json', 'w') as f:
    json.dump(curr_ammount, f)

In [5]:
with open('curr_ammount.json', 'r') as f:
    curr_ammount = json.load(f)
curr_ammount

{'USDCUSDT': 10,
 'BCHUSDT': 0,
 'BTCUSDT': 0,
 'DOGEUSDT': 0,
 'ETHUSDT': 0,
 'LTCUSDT': 0,
 'CRVUSDT': 0,
 'FTMUSDT': 0,
 'LINKUSDT': 0,
 'SOLUSDT': 0}

In [6]:
PUBLIC_API_KEY = "account-dnvGW5r4z8mY9ATNERjU"
PRIVATE_API_KEY = "BWvsWYXsKNcpF4i78Ni3Sqio6kY"
Client = gemini.PrivateClient(PUBLIC_API_KEY, PRIVATE_API_KEY)

In [7]:
syms = ['USDCUSDT', 'BCHUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'LTCUSDT', 'CRVUSDT', 'FTMUSDT', 'LINKUSDT', 'SOLUSDT']

In [8]:
new_syms = ['usd'] + [x.split('USDT')[0].lower() for x in syms if x!='USDCUSDT']
new_syms

['usd', 'bch', 'btc', 'doge', 'eth', 'ltc', 'crv', 'ftm', 'link', 'sol']

In [9]:
N = len(syms)

In [10]:
tradable = set(Client.symbols())
pairs = []
for i in range(N-1):
    for j in range(i+1, N):
        from_ = new_syms[i]
        to_ = new_syms[j]
        if from_+to_ in tradable:
            pairs.append((from_, to_))
        elif to_+from_ in tradable:
            pairs.append((to_, from_))

In [11]:
data = pd.DataFrame(index=new_syms, columns=new_syms)
for from_, to_ in pairs:
    market_data = Client.get_ticker(from_+to_)
    data.loc[from_, to_] = float(market_data['bid'])
    data.loc[to_, from_] = 1/float(market_data['ask'])

In [19]:
INFINITY = np.inf
EPSILON = 1e-10

na_matrix = data*INFINITY
na_matrix.fillna(0, inplace = True)
na_matrix = na_matrix.values

data.fillna(1, inplace = True)

curr_ammount_vector = np.array([curr_ammount[x] for x in syms])
target_weight_vector = np.array([target_weight[x] for x in syms])

In [20]:
model = ro.Model('minimize fee')

X = model.dvar((N, N))
Y = model.dvar(N)

model.min(X.sum(axis=1) @ data['usd'].values)

model.st(abs(Y * data['usd'].values - target_weight_vector * (Y @ data['usd'].values)) <= EPSILON)

model.st(Y == curr_ammount_vector - X.sum(axis=1) + (X * data.values).sum(axis=0))
model.st(X.sum(axis=1) <= curr_ammount_vector)

model.st(X <= na_matrix)
model.st(X >= 0)

In [21]:
model.solve(grb)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0215s


In [24]:
X.get()

array([[0.        , 0.99987125, 0.99945363, 0.99980395, 0.99947317,
        0.99977573, 1.00059359, 0.99995324, 1.00187864, 0.99998403],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [16]:
data

,usd,bch,btc,doge,eth,ltc,crv,ftm,link,sol
usd,1.00000,0.003136,0.000026,7.391529,0.000343,0.009385,0.475873,0.925669,0.075188,0.010284
bch,318.65000,1.000000,0.008180,1.000000,0.108700,2.439024,1.000000,1.000000,1.000000,1.000000
btc,38949.59000,121.951220,1.000000,287356.321839,13.352918,364.963504,1.000000,1.000000,2926.972047,1.000000
doge,0.13521,1.000000,0.000003,1.000000,0.000046,1.000000,1.000000,1.000000,1.000000,1.000000
eth,2915.94000,9.025271,0.074850,21547.080371,1.000000,27.322404,1.000000,1.000000,217.244900,1.000000
ltc,106.49000,0.240000,0.002730,1.000000,0.036400,1.000000,1.000000,1.000000,1.000000,1.000000
crv,2.09850,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
ftm,1.07950,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
link,13.26461,1.000000,0.000340,1.000000,0.004549,1.000000,1.000000,1.000000,1.000000,1.000000
sol,97.16300,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
